In [84]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import BaseScheduler
from mesa.space import MultiGrid
import numpy as np
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
import random
import math

In [89]:


def find_bikes_req(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req =math.ceil(((num_boxes)/(b_parcels+(v_parcels*ratio_v_b))))
    return b_req

def find_bikes_dec(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req =(((num_boxes)/(b_parcels+(v_parcels*ratio_v_b))))
    return b_req

def find_vans_req(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req = find_bikes_dec(num_boxes,ratio_v_b,v_parcels,b_parcels)
    v_req= math.ceil(b_req*ratio_v_b)
    return v_req



In [3]:
Total_Parcels=100
Bike_Capacity=2
Van_Capacity=10
Ratio_of_vans_to_bikes=1
#is used to calcualte other vehicles by multiplyig the variable by the sum of vans and bikes
Traffic_Level_multiplier=1
Total_Bikes=find_bikes_req(Total_Parcels,Ratio_of_vans_to_bikes,Van_Capacity,Bike_Capacity)
Total_Vans=find_vans_req(Total_Parcels,Ratio_of_vans_to_bikes,Van_Capacity,Bike_Capacity)
Total_Cars=Traffic_Level_multiplier*(Total_Bikes+Total_Vans)
Parcel_list=[*range(0,Total_Parcels)]

[1, 2, 4, 6] [9, 6, 3]


In [100]:
#AGENTS
#mess with this code for experimental agents
#this was a defult agent 
class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        neighbors = []
        x, y = self.pos
        for dx in [0, 1]:
            for dy in [0]:
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other_agent = self.random.choice(cellmates)
            other_agent.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

#this agent is a potential parking location at the curb            
class curb_border(Agent):
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.delivered=[]
        self.tick=1
        self.destination_for=[]
        self.bike_del=[]
        #chance is the probability of a vacant space for parking
        chance=random.randint(0, 10)
        if chance==0:
            self.available = True
        else:
            self.available = False     
       
        
    

    def step(self):
        return
        
 #building agent/ delivery desitinations       
class building(Agent):
    def __init__(self, pos, model,boxes):
        super().__init__(pos, model)
        self.tick=1
        self.delivered=[]
        self.waiting_for=boxes
        self.val=0
        chance=random.randint(0, 3)
        if chance==0:
            self.destination = True
        else:
            self.destination = False
        
        self.in_range=[]
        self.loading_bays = []
        self.pos=pos
        x, y = self.pos
        
        #insert positions as argument
        def get_curb(self, pos):
            this_cell = self.model.grid.get_cell_list_contents([pos])
            for agent in this_cell:
                if type(agent) is curb_border:
                    return agent
        
        #i in the following should return cood tuple (x,y) and gets all surrounding cells
    
    
    
    
    
    def distribute(self):
        
        def get_curve(bib):
            
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
            
        for i in self.model.grid.get_neighborhood(self.pos, moore=True, radius=3):
            
        #keeping values equal to those on grid
            
            if i[0]<31 and i[1]==1:
                self.in_range.append(get_curve(i))
        
        for c in self.in_range:
            if c. available == True:
                self.loading_bays.append(c)                
                    
        for p in self.loading_bays:
            p.destination_for=self.waiting_for+p.destination_for
            
    def distribute_for_bike(self):
        
        def get_curve(bib):
            
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
        
        neighborz = []
        x, y = self.pos
        for dx in [0]:
            for dy in [-1]:
                neighborz.append((x+dx, y+dy))
        
        get_curve(neighborz[0]).bike_del= self.waiting_for+get_curve(neighborz[0]).bike_del     
        
        #agent below given as bike dest              
    

    
    def step(self):
        if self.val==0:
            self.distribute()
            self.distribute_for_bike()
            self.val=self.val+1
        
        
    
class bikes(Agent):
    """ An ebike agent """
    
    def __init__(self, unique_id, model,boxes):
        super().__init__(unique_id, model)
        #state: 1= drving ,0= parked/delivering 
        self.state = 1
        self.parcels=boxes
        self.storage=[]
        self.delivered=[]
    
    def move(self):
        print("trying to move")
        neighbors = []
        x, y = self.pos
        for dx in [1]:
            for dy in [0]:
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        if possible_steps[0][0]<30:
            if len(self.model.grid.get_cell_list_contents([possible_steps[0]]))==0:
                new_position = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, new_position)
        else:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)
    
    def get_curb(self):
        
        for i in self.model.grid.get_cell_list_contents([self.pos]):
            if type(i) is curb_border:
                return i
    
    def park(self):
        
        
        neighbors = []
        x, y = self.pos
        
        for dy in [1]:
            
            for dx in [0]:
                
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        if self.state==1:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)
            self.state=0
        
    
    
    
    def check_arrival(self):
        
        
        
        nerds = []
        x, y = self.pos
        for dx in [0]:
            for dy in [1]:
                nerds.append((x+dx, y+dy))
        
        def get_curve(bib):
            print(bib)
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
                
        location=get_curve(nerds[0])
        
        for b in self.parcels:
            for j in location.bike_del:
                if b==j:
                    self.timer=0
                    self.park()
                    
    
    
    
    def deliver(self):
        self.timer=self.timer+1
        print("now delivering")
        
        def get_curve(bib):
            
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
        
        for b in get_curve(self.pos).bike_del :
            for j in  self.parcels:
                if b==j:
                    self.parcels.remove(b)
                    self.delivered.append(b)
                    
                    
    def depart(self):
        print("now departing")
        neighbors = []
        x, y = self.pos
        
        for dy in [-1]:
            
            for dx in [0]:
                
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        self.state=1
        
    
    #parcel list = coods of buildings random from list of buildings       
    #delivery destinations = list of buildings' loading bays attributes
    
        
        
        
         

    def step(self):
        print("im a bike and im stepping")
        
        if self.state==1:
            print("trying a step")
            self.move()
            self.check_arrival()
        else:
            print("trying someting else")
            self.deliver()
            if self.timer>10:
                self.depart()
        
     
        
        
        #if state is parked 
           #if time calculated >0
              #time calculated= time calculated-1
           #else 
               #move out
               #change state to driving
    
    
"""     def look_for_parking(self):
        loading_bays = []
        x, y = self.pos
        self.vison= 3 #how far van driver is willing to park from destination
        #i in the following should return cood tuple (x,y) and gets all surrounding cells
        for i in self.model.grid.get_neighborhood(self.pos, self.moore,False, radius=self.vision):
            #limits parking to curbs (so roads are not parking)
            if type_distribution[i]==3:
                #makes sure the bay is not occupied by another van
                if len(self.model.grid.get_cell_list_contents([i]))<2 and self.model.grid.get_cell_list_contents([i]).destination == true:
                    loading_bays.append(i)
        return loading_bays        
"""            
    #self.list_of_correct_spaces=look_for_parking()
      
        ##test this and probably move to model setup or check if you can activate an agent's function
        
    
     
        
class vans(Agent):
    """ A van agent """
    
    def __init__(self, unique_id, model,boxes):
        super().__init__(unique_id, model)
        #state: 1= drving ,0= parked/delivering 
        self.state = 1
        self.parcels=boxes
        self.storage=[]
        self.delivered=[]
    
    def move(self):
        
        neighbors = []
        x, y = self.pos
        for dx in [1]:
            for dy in [0]:
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        if possible_steps[0][0]<30:
            if len(self.model.grid.get_cell_list_contents([possible_steps[0]]))==0:
                new_position = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, new_position)
        else:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)
    
    
    
    def get_curb(self):
        
        for i in self.model.grid.get_cell_list_contents([self.pos]):
            if type(i) is curb_border:
                return i
    
    def park(self):
        
        neighbors = []
        x, y = self.pos
        
        for dy in [1]:
            
            for dx in [0]:
                
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        self.state=0
        self.deliver()
    
    
    
    def check_arrival(self):
        
        
        
        nerds = []
        x, y = self.pos
        for dx in [0]:
            for dy in [1]:
                nerds.append((x+dx, y+dy))
        
        def get_curve(bib):
            print(bib)
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
                
        location=get_curve(nerds[0])
        
        for b in self.parcels:
            for j in location.destination_for:
                
                if b==j and len(self.model.grid.get_cell_list_contents([location.pos]))==1:
                    self.timer=0
                    self.park()
                    
    
    
    
    def deliver(self):
        
        print("now delivering")
        
        def get_curve(bib):
            print(bib)
            for i in self.model.grid.get_cell_list_contents([bib]):
                if type(i) is curb_border:
                    return i
        
        
        
        for b in get_curve(self.pos).destination_for :
            for j in  self.parcels:
                if b==j:
                    self.parcels.remove(b)
                    self.delivered.append(b)
        
        
    def depart(self):
        print("now departing")
        neighbors = []
        x, y = self.pos
        
        for dy in [-1]:
            
            for dx in [0]:
                
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        new_position = self.random.choice(possible_steps)
        if len(self.model.grid.get_cell_list_contents([new_position]))==0:
            self.model.grid.move_agent(self, new_position)
            self.state=1
        
        
        
    
    #parcel list = coods of buildings random from list of buildings       
    #delivery destinations = list of buildings' loading bays attributes
    
        
        
        
         

    def step(self):
        print("im a van stepping")
        if self.state==1:
            self.move()
            self.check_arrival()
        else:
            self.timer=self.timer+1
            if self.timer>10:
                self.depart()    
        

In [114]:
#Model
#mess with this

class VanByke(Model):
    """A model with some number of agents."""
    
    def __init__(self,width, height, N=12,BC=2,VC=10,R=1,T=1):
        super().__init__()
        #self.num_agents = 1
        self.Total_Parcels=N
        self.Bike_Capacity=BC
        self.Van_Capacity=VC
        self.Ratio_of_vans_to_bikes=R
        #is used to calcualte other vehicles by multiplyig the variable by the sum of vans and bikes
        self.Traffic_Level_multiplier=T
        self.Total_Bikes=find_bikes_req(self.Total_Parcels,self.Ratio_of_vans_to_bikes,self.Van_Capacity,self.Bike_Capacity)
        self.Total_Vans=find_vans_req(self.Total_Parcels,self.Ratio_of_vans_to_bikes,self.Van_Capacity,self.Bike_Capacity)
        self.Total_Cars=Traffic_Level_multiplier*(self.Total_Bikes+self.Total_Vans)
        self.Parcel_list=[*range(0,self.Total_Parcels)]
        self.grid = MultiGrid(width, height, True)
        self.place=0
        self.all_vehicles=[]
        self.schedule = RandomActivation(self)
        #self.schedule =BaseScheduler(self)
        type_distribution = np.genfromtxt("road_map.txt")
        
        #create curb
        for _,x, y in self.grid.coord_iter():
            if type_distribution[x,y]==3: #3 is the chosen identifier for curb side in the txt doc
                new_spot=curb_border((x,y),self)
                
                self.grid.place_agent(new_spot,(x, y))
                self.schedule.add(new_spot)
        
        #create buildings
        for _,x, y in self.grid.coord_iter():
            if type_distribution[x,y]==4: #4 is the chosen identifier for buildings in the txt doc
                
                new_building=building((x,y),self,[self.Parcel_list[random.randint(0, self.Total_Parcels-1)]])
                #print(self.loading_bays)
                
                
                self.grid.place_agent(new_building,(x, y))
                self.schedule.add(new_building)
                
                
        #check for curbs around if parking is possible,
        #possible when agent present is a curb and attribute availble == true
        
       #create bikes 
        
        
        """for i in range(self.Total_Bikes):
            nit=[]
            
            for l in range(self.Bike_Capacity):
                if len(self.Parcel_list)>0:
                
                    nit.append(self.Parcel_list[-1])
                    self.Parcel_list.pop()
            a=bikes(i, self, nit)
            
            self.all_vehicles.append(a)"""
            
    
    
    
    
    
    #create vans
        
        
        for i in range(self.Total_Vans):
            nit=[]
            for l in range(self.Van_Capacity):
                if len(self.Parcel_list)>0:
                
                    nit.append(self.Parcel_list[-1])
                    self.Parcel_list.pop()
            b=vans(i, self, nit)
            
            self.all_vehicles.append(b)
        
        
        #create bikes 
        
        
        for i in range(self.Total_Bikes):
            nit=[]
            
            for l in range(self.Bike_Capacity):
                if len(self.Parcel_list)>0:
                
                    nit.append(self.Parcel_list[-1])
                    self.Parcel_list.pop()
            a=bikes(i, self, nit)
            
            self.all_vehicles.append(a)
            
           
             
          
            
        
        
        
        """        
        # Create agents""""""
        for i in range(self.num_agents):
            a=vans(i, self)
            #a=bikes(i, self)
            #a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = 0
            self.grid.place_agent(a, (x, y))
        """            
        self.running = True
    
       
       
      
    def step(self):
        print(self.all_vehicles)
        self.schedule.step()
        print("stepping in model")
        if len(self.all_vehicles)>0 and [*range(0,2000)][self.place]%2==0:
            
                self.schedule.add(self.all_vehicles[-1])
                self.grid.place_agent(self.all_vehicles[-1],(0,0))
                self.all_vehicles.pop()
                print("placing")
        
        self.place+=1        
          
            

In [7]:
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {}

    if type(agent) is MoneyAgent :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, }

        if agent.wealth > 3:
            portrayal["Color"] = "red"
            portrayal["Layer"] = 1
        elif agent.wealth > 2:
            portrayal["Color"] = "orange"
            portrayal["r"] = 0.4
            portrayal["Layer"] = 1
        elif agent.wealth > 1:
            portrayal["Color"] = "yellow"
            portrayal["r"] = 0.3
            portrayal["Layer"] = 1  
        elif agent.wealth > 0:
            portrayal["Color"] = "green"
            portrayal["r"] = 0.2
            portrayal["Layer"] = 1     
        else:
            portrayal["Color"] = "grey"
            portrayal["Layer"] = 1
            portrayal["r"] = 0.1

    if type(agent) is bikes :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, "Color" : "yellow", "Layer":1,}
    
    
    if type(agent) is vans :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, "Color" : "grey", "Layer":1,}
      
    elif type(agent) is curb_border:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true",
                "Layer": 0 , "w":1 , "h":1,}
        
        if agent.available== True:
            portrayal["Color"] = "green"
        else:
            portrayal["Color"] = "red" 
            
            
        
    elif type(agent) is building:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true",
                "Layer": 0 , "w":1 , "h":1,}
        
        if agent.destination== True:
            portrayal["Color"] = "blue"
        else:
            portrayal["Color"] = "grey"     
    

    return portrayal
grid = CanvasGrid(agent_portrayal, 30, 3, 1200, 120)

In [115]:

harris=harris+1
server = ModularServer(VanByke,
                       [grid],
                       "VanByke Model",
                       { "width":30, "height":3})
server.port = harris# The default
server.launch()


Interface starting at http://127.0.0.1:9511


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
[<__main__.vans object at 0x7ff91a645850>, <__main__.bikes object at 0x7ff91a6455d0>]
stepping in model
placing
{"type":"get_step","step":2}
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":3}
im a bike and im stepping
trying someting else
now delivering
stepping in model
placing
{"type":"get_step","step":4}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":5}
[]
im a van stepping
(2, 1)
stepping in model
{"type":"get_step","step":6}
[]
im a van stepping
(3, 1)
stepping in model
{"type":"get_step","step":7}
[]
im a van stepping
(4, 1)
now delivering
(4, 1)
stepping in model
{"type":"get_step","step":8}
[]
im a van stepping
stepping in model
{"type":"get_step","step":9}
[]
im a van stepping
stepping in model
{"type":"get_step","step":10}
[]
im a van stepping
stepping in model
{"type":"get_step","step":11}
[]
im a van stepping
stepping in model
{"typ

{"type":"get_step","step":108}
[]
im a van stepping
(4, 1)
stepping in model
{"type":"get_step","step":109}
[]
im a van stepping
(5, 1)
stepping in model
{"type":"get_step","step":110}
[]
im a van stepping
(6, 1)
stepping in model
{"type":"get_step","step":111}
[]
im a van stepping
(7, 1)
stepping in model
{"type":"get_step","step":112}
[]
im a van stepping
(8, 1)
stepping in model
{"type":"get_step","step":113}
[]
im a van stepping
(9, 1)
stepping in model
{"type":"get_step","step":114}
[]
im a van stepping
(10, 1)
stepping in model
{"type":"get_step","step":115}
[]
im a van stepping
(11, 1)
stepping in model
{"type":"get_step","step":116}
[]
im a van stepping
(12, 1)
stepping in model
{"type":"get_step","step":117}
[]
im a van stepping
(13, 1)
stepping in model
{"type":"get_step","step":118}
[]
im a van stepping
(14, 1)
stepping in model
{"type":"get_step","step":119}
[]
im a van stepping
(15, 1)
stepping in model
{"type":"get_step","step":120}
[]
im a van stepping
(16, 1)
stepping i

{"type":"get_step","step":11}
[]
im a van stepping
(8, 1)
stepping in model
{"type":"get_step","step":12}
[]
im a van stepping
(9, 1)
stepping in model
{"type":"get_step","step":13}
[]
im a van stepping
(10, 1)
now delivering
(10, 1)
stepping in model
{"type":"get_step","step":14}
[]
im a van stepping
stepping in model
{"type":"get_step","step":15}
[]
im a van stepping
stepping in model
{"type":"get_step","step":16}
[]
im a van stepping
stepping in model
{"type":"get_step","step":17}
[]
im a van stepping
stepping in model
{"type":"get_step","step":18}
[]
im a van stepping
stepping in model
{"type":"get_step","step":19}
[]
im a van stepping
stepping in model
{"type":"get_step","step":20}
[]
im a van stepping
stepping in model
{"type":"get_step","step":21}
[]
im a van stepping
stepping in model
{"type":"get_step","step":22}
[]
im a van stepping
stepping in model
{"type":"get_step","step":23}
[]
im a van stepping
stepping in model
{"type":"get_step","step":24}
[]
im a van stepping
now dep

{"type":"get_step","step":83}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":84}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":85}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":86}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":87}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"reset"}
{"type":"get_step","step":1}
[<__main__.vans object at 0x7ff91a74a3d0>, <__main__.bikes object at 0x7ff91a74af10>]
stepping in model
placing
{"type":"get_step","step":2}
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":3}
im a bike and im stepping
trying a step
trying to move
(2, 1)
stepping in model
placing
{"type":"get_step","step":4}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":5}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":6}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"ge

{"type":"get_step","step":45}
[]
im a van stepping
(20, 1)
stepping in model
{"type":"get_step","step":46}
[]
im a van stepping
(21, 1)
stepping in model
{"type":"get_step","step":47}
[]
im a van stepping
(22, 1)
stepping in model
{"type":"get_step","step":48}
[]
im a van stepping
(23, 1)
stepping in model
{"type":"get_step","step":49}
[]
im a van stepping
(24, 1)
stepping in model
{"type":"get_step","step":50}
[]
im a van stepping
(25, 1)
stepping in model
{"type":"get_step","step":51}
[]
im a van stepping
(26, 1)
stepping in model
{"type":"get_step","step":52}
[]
im a van stepping
(27, 1)
stepping in model
{"type":"get_step","step":53}
[]
im a van stepping
(28, 1)
stepping in model
{"type":"get_step","step":54}
[]
im a van stepping
(29, 1)
stepping in model
{"type":"get_step","step":55}
[]
im a van stepping
(0, 1)
stepping in model
{"type":"get_step","step":56}
[]
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":57}
[]
im a van stepping
(2, 1)
stepping in model
{"

SyntaxError: invalid syntax (<ipython-input-107-ea891ba5c652>, line 1)

{"type":"reset"}
{"type":"get_step","step":1}
[<__main__.bikes object at 0x7ff91a6b5090>, <__main__.vans object at 0x7ff91a6b5d90>]
stepping in model
placing
{"type":"get_step","step":2}
im a van stepping
(1, 1)
stepping in model
{"type":"get_step","step":3}
im a van stepping
(2, 1)
stepping in model
placing
{"type":"get_step","step":4}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":5}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":6}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":7}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":8}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step","step":9}
[]
im a bike and im stepping
trying a step
trying to move
(1, 1)
stepping in model
{"type":"get_step"